# ESM tutorial

This tutorial explains how to use the effective screening medium (ESM) method implemented in theUANTUM ESPRESSO distribution. 
This tutorial consists of the following two parts.  

* Example 1: Water molecule  
    First, we learn the fundamental option via the calculation of an isolated molecule. 

* Example 2: Al(111) surface  
    Next, we learn the application of ESM method for the material surface.  
    
Throughout the tutorial, we use Atomic Simulation Enviroment (ASE: https://wiki.fysik.dtu.dk/ase/), which is a Python library for setting up, manipulating, running, visualizing and analyzing atomistic simulations and plane-wave basis sets within pseudopotential framework code QUANTUM ESPRESSO distribution (https://www.quantum-espresso.org).   
First of all, we should download the ASE package of Minoru Otani version from his GitLab (https://gitlab.com/minoru-otani/ase) and QUANTUM ESPRESSO distribution (QE) from https://www.quantum-espresso.org. 
After downloading, you should install the ASE and QE.  
If you have the cluster machine or super computer account, accessing to them with the SSH,  
QE should be compiled on the remote hosts.  

### NOTE
* The installation guide for the ASE package can be found on the ASE website (https://wiki.fysik.dtu.dk/ase/install.html).  
* PWscf and PP codes are needed to try this tutorial.

After installation, run the following two Python scripts on Jupyter notebook: 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

In [2]:
import os
import time
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import qeutil
import xml.etree.ElementTree as ET
from qeutil import extract_structure_data, write_qe_input, qe_bool, parse_qe_xml, plot_esm1
from ase import Atoms
from ase.data import atomic_masses, atomic_numbers
from ase.io.cube import read_cube_data
from ase.visualize import view

Now, we finish the preparation for the start of the tutorials. 
In the next section, we start the Example 1. 

# Example 1: Water molecule

Here, we will execute the two calculations:  
* Water molecule under the periodic boundary condition (PBC)  
* Water molecule using ESM method  



## Water molecule (periodic boundary condition)

Here, we recall the drawback of the PBolar-molecule and learn how to use this tutorial.  
C calculation for polar molecules
First, we calculate H$_2$O molecule using usual DFT calculation under the PBC.  

### Determination of project and job information
Run the following two Python scripts on your Jupyter Notebook. 

In [3]:
project = 'H2O_pbc'
prj_info = {
    'project': project,
    'projdir': '/Users/otani/Program/q-e',
    'prefixes': ['H2O_pbc'],
}
prj_info['bindir'] = f"{prj_info['projdir']}/bin"
prj_info['pseudodir'] = f"{prj_info['projdir']}/pseudo"
prj_info['outdir'] = "./out"

In [4]:
job_info = {
    'node':     1,   
    'mpinum':   28,  
    'ompnum':   1,
    'cputime':  dt.timedelta(days=0, hours=0, minutes=30),
    'para_img': 1,
    'para_kpt': 1,
    'para_tsk': 1,
    'para_bnd': 1,
}

The first script determines the project name, position of home directry (local host), and prefix of files,  
which corresponds to the `prefix` of PWscf inputs, respectively.  
The second script determines job-running information.  
* `node`: request computational nodes
* `mpinum`: the number of mpi procs.
* `ompnum`: the number of omp threads.
* `cputime`: corresponding to the `max_seconds` value of PWscf inputs. 
* `para_*`: respectively corresponding to number of divisions for MPI parallel running of PWscf. 

After runnning above scripts, `program` directry is created below the `/Users/username/`.  
The results of this tutorial is summarized in the program directry. 

### Make input file for PWscf using ASE

Thanks to the ASE package, after runnig the following python script, input files for the PWscf is autmatically generated. 

In [5]:
atoms = Atoms('OH2', [(5.00000000, 5.0000000000, 0.00000000),
                      (5.75754080, 5.0000000000, 0.58707964),
                      (4.24245910, 5.0000000000, 0.58707964)], 
             cell=[(10.0000000000,  0.0000000000,  0.0000000000),
                   ( 0.0000000000, 10.0000000000,  0.0000000000),
                   ( 0.0000000000,  0.0000000000, 10.0000000000)],
              pbc=(True, True, True)
)

input_data = {
    'control': {
        'calculation'        : 'scf',
        'restart_mode'       : 'from_scratch',
        'prefix'             : '',
        'max_seconds'        : job_info['cputime'].total_seconds(),
        'pseudo_dir'         : prj_info['pseudodir'],
        'outdir'             : prj_info['outdir'],
        'tprnfor'            : qe_bool(True),
    },
    'system': {
        'ibrav'              : 0,
        'nat'                : len(atoms),
        'ntyp'               : len(set(atoms.get_chemical_symbols())),
        'ecutwfc'            : 25.0, 
        'ecutrho'            : 200.0,
        'occupations'        : 'fixed', 
        'nbnd'               : 20,
        'assume_isolated'    : 'esm',
        'esm_bc'             : 'pbc',
    },
    'electrons': {
        'mixing_beta'        : 0.5,
    }
}

input_data_chng = {
    prj_info['prefixes'][0]: {
        'control': {
            'prefix'     : prj_info['prefixes'][0],
        }
    },
}
structure_data = extract_structure_data(atoms)
pseudopotentials = {'H':'H.pbe-van_ak.UPF', 'O':'O.pbe-van_ak.UPF'}
atomic_species = {
    elem: (atomic_masses[atomic_numbers[elem]], pseudopotentials[elem])
    for elem in pseudopotentials
}
inpfile = 'test.in'
for prefix in prj_info['prefixes']:
    inpfile = f"{prefix}.in"  # 各 prefix ごとに inpfile を作成
    for group in input_data_chng[prefix]:
        for name, value in input_data_chng[prefix][group].items():
            input_data[group][name] = value
    write_qe_input(input_data, atomic_species, structure_data, filename=inpfile)

Quantum ESPRESSO input file 'H2O_pbc.in' has been generated.


### View unit cell 

Following script desplay the atomic structure in your unit cell.  
Please, check the unit cell after running the scripyt.

In [ ]:
view(atoms, viewer='ngl')

### Print out the results of SCF calculation written in `H2O.out`

Here, we print out Total energy, forces and total number of charge described in `H2O.out`. 

In [ ]:
# `prj_info` から prefix リストと出力ディレクトリを取得
prefixes = prj_info['prefixes']
outdir = prj_info['outdir']

# Quantum ESPRESSO の XML 解析
scf_result = parse_qe_xml(prefixes, outdir)

# Print all results from XML
for prefix in prj_info['prefixes']:
    print('============================================')
    print(f"Prefix: {prefix}")
    print('============================================')
    # 全エネルギー (eV)
    energy = scf_result[prefix]['energy']
    print(f"Total Energy (eV): {energy:.6f}")
    # フェルミエネルギー (eV)
    fermi_energy = scf_result[prefix]['fermi_energy']
    if fermi_energy is not None:
        print(f"Fermi Energy (eV): {fermi_energy:.6f}")
    else:
        print("Fermi Energy: Not found")
    # 余剰電荷
    tot_charge = scf_result[prefix]['tot_charge']
    if tot_charge is not None:
        print(f"Total Charge: {tot_charge:.6f}")
    else:
        print("Total Charge: Not found")
    # 原子の力を取得して表示
    forces = scf_result[prefix]['forces']
    if forces:  # フォースデータが空でない場合のみ出力
        print('Forces acting on atoms (eV/Å)')
        for atom_id, fx, fy, fz in forces:
            print(f"  Atom {atom_id}: ({fx:.6f}, {fy:.6f}, {fz:.6f})")
    else:
        print("Forces: Not found")
    print('---------------------------------------------')

In [ ]:
plot_esm1(prj_info['outdir']+'/'+prj_info['prefixes'][0]+'.esm1',titlename='H2O bc1')

## Water molecule ('bc1' boundary condition)

Here, we learn how to use the ESM calculation with the "Vacuum/Cell/Vacuum" boundary condistion.  
We find the new two inputs from following python scripts:

* `assume_isolated`  
* `esm_bc`  

First of all, check the meanings of these input values at the Web site of input file discription for PWscf code.  
The boundary condition of Vacuum/Cell/Vacuum indicates that the z-direction of the unit-cell is sandwiched by infinite vacuums.  
Thus, we treat unitcell is the mix boundary condition:  
* xy-plane is treaeted under the PBC.  
* z-direcion is treated as the open-boundary.  

The theoretical back-ground of ESM technique can be found in elsewhere (M. Otani and O. Sugino, PRB 73, 115407 (2006).).  
Then, run the following python scripts.

In [ ]:
project = 'H2O_bc1'
prj_info = {
    'project': project,
    'projdir': '/Users/otani/Program/q-e',
    'prefixes': ['H2O_bc1'],
}
prj_info['bindir'] = f"{prj_info['projdir']}/bin"
prj_info['pseudodir'] = f"{prj_info['projdir']}/pseudo"
prj_info['outdir'] = "./out"

In [ ]:
atoms = Atoms('OH2', [(5.00000000, 5.0000000000, 0.00000000),
                      (5.75754080, 5.0000000000, 0.58707964),
                      (4.24245910, 5.0000000000, 0.58707964)], 
             cell=[(10.0000000000,  0.0000000000,  0.0000000000),
                   ( 0.0000000000, 10.0000000000,  0.0000000000),
                   ( 0.0000000000,  0.0000000000, 10.0000000000)],
)

input_data = {
    'control': {
        'calculation'        : 'scf',
        'restart_mode'       : 'from_scratch',
        'prefix'             : '',
        'max_seconds'        : job_info['cputime'].total_seconds(),
        'pseudo_dir'         : prj_info['pseudodir'],
        'outdir'             : prj_info['outdir'],
        'tprnfor'            : qe_bool(True),
    },
    'system': {
        'ibrav'              : 0,
        'nat'                : len(atoms),
        'ntyp'               : len(set(atoms.get_chemical_symbols())),
        'ecutwfc'            : 25.0, 
        'ecutrho'            : 200.0,
        'occupations'        : 'fixed', 
        'nbnd'               : 20,
        'assume_isolated'    : 'esm',
        'esm_bc'             : 'bc1',
    },
    'electrons': {
        'mixing_beta'        : 0.5,
    }
}

input_data_chng = {
    prj_info['prefixes'][0]: {
        'control': {
            'prefix'     : prj_info['prefixes'][0],
        }
    }
}
structure_data = extract_structure_data(atoms)
pseudopotentials = {'H':'H.pbe-van_ak.UPF', 'O':'O.pbe-van_ak.UPF'}
atomic_species = {
    elem: (atomic_masses[atomic_numbers[elem]], pseudopotentials[elem])
    for elem in pseudopotentials
}
for prefix in prj_info['prefixes']:
    inpfile = f"{prefix}.in"  # 各 prefix ごとに inpfile を作成
    for group in input_data_chng[prefix]:
        for name, value in input_data_chng[prefix][group].items():
            input_data[group][name] = value
            #write_qe_input(input_data, atomic_species, structure_data, filename=inpfile, kpts=(1, 1, 1), koffset=(0,0,0))
    write_qe_input(input_data, atomic_species, structure_data, filename=inpfile)

In [ ]:
# `prj_info` から prefix リストと出力ディレクトリを取得
prefixes = prj_info['prefixes']
outdir = prj_info['outdir']

# Quantum ESPRESSO の XML 解析
scf_result = parse_qe_xml(prefixes, outdir)

# Print all results from XML
for prefix in prj_info['prefixes']:
    print('============================================')
    print(f"Prefix: {prefix}")
    print('============================================')
    # 全エネルギー (eV)
    energy = scf_result[prefix]['energy']
    print(f"Total Energy (eV): {energy:.6f}")
    # フェルミエネルギー (eV)
    fermi_energy = scf_result[prefix]['fermi_energy']
    if fermi_energy is not None:
        print(f"Fermi Energy (eV): {fermi_energy:.6f}")
    else:
        print("Fermi Energy: Not found")
    # 余剰電荷
    tot_charge = scf_result[prefix]['tot_charge']
    if tot_charge is not None:
        print(f"Total Charge: {tot_charge:.6f}")
    else:
        print("Total Charge: Not found")
    # 原子の力を取得して表示
    forces = scf_result[prefix]['forces']
    if forces:  # フォースデータが空でない場合のみ出力
        print('Forces acting on atoms (eV/Å)')
        for atom_id, fx, fy, fz in forces:
            print(f"  Atom {atom_id}: ({fx:.6f}, {fy:.6f}, {fz:.6f})")
    else:
        print("Forces: Not found")
    print('---------------------------------------------')

In [ ]:
plot_esm1(prj_info['outdir']+'/'+prj_info['prefixes'][0]+'.esm1',titlename='H2O bc1')

Here, we obtain the results of rho, V_hartree, V_ion and V_electrostatic with ESM correction.  
Thanks to "Vacuum/Slab/Vacuum" condition, interaction between image cells is cancelled out and flat vacuum level can be found in V_electrostatic.  
In addition, energy level zero under the OBC is defined as midpoint between $V(z=-\infty)$ and $V(z=+\infty)$, where $V(z)$ is the electrostatic potential. 


In [ ]:
#v = nv.show_ase(atoms)
#v.add_ball_and_stick()
#v.add_unitcell()
#v
temp = atoms
print(atoms.get_cell())
temp.translate((0,0,-(np.sum(temp.positions[:,2]))/temp.positions.shape[0]))
view(atoms, viewer='ngl')

# Example 2: Al(111) surface

Here, we learn how to use the ESM calculation for the surface.  
As for example, we use Al(111) surface using repeated slab model having four atomic layers.  

Hereafter, we will use following three types of ESM boundary conditions:  

* `bc1`: Vacuum/Slab/Vacuum (VSV)
* `bc2`: Metal/Slab/Metal (MSM)  
* `bc3`: Vacuum/Slab/Metal (VSM)



## Al(111) surface ('bc1' boundary condition)

First, we use the VSV model. The VSV model is already used in the previous exampe.  This model is useful for ordinal slab calculation with thin vacuum layer, because, in this model, z-direction of surface slab, which corresponds to the surface normal-direction, is treated as open boundary condition (OBC). 

Run following scripts, and calculate electronic structure of Al(111) surface with VSV model. 

In [ ]:
project = 'Al-slab'
prj_info = {
    'project': project,
    'projdir': '/Users/otani/Program/q-e',
    'prefixes': ['Al0'],
}
prj_info['bindir'] = f"{prj_info['projdir']}/bin"
prj_info['pseudodir'] = f"{prj_info['projdir']}/pseudo"
prj_info['outdir'] = "./out"

### k-point parallization

Since the k-point sampling is required for the slab calculation, we use k-point parallization mode hereafter.  
Following a set of `job_info` is a sample for using k-parallel running of PWscf.  

In [ ]:
job_info = {
    'node':     4,   
    'mpinum':   28,  
    'ompnum':   1,
    'cputime':  dt.timedelta(days=0, hours=0, minutes=30),
    'para_img': 1,
    'para_kpt': 4,
    'para_tsk': 1,
    'para_bnd': 1,
}

In [ ]:
atoms = Atoms('Al4', [(2.85595455,  0.00000000,  3.49781569),
                      (1.42797727,  0.82444307,  1.16593856),
                      (2.85595454,  1.64888613, -1.16593856),
                      (0.00000000,  0.00000000, -3.49781569)], 
             cell=[( 2.85595455,  0.0000000000,  0.0000000000),
                   ( 1.42797727,  2.4733291965,  0.0000000000),
                   ( 0.00000000,  0.0000000000, 22.9956313827)],
              pbc=(True, True, False)
)

input_data = {
    'control': {
        'calculation'        : 'scf',
        'restart_mode'       : 'from_scratch',
        'prefix'             : '',
        'max_seconds'        : job_info['cputime'].total_seconds(),
        'pseudo_dir'         : prj_info['pseudodir'],
        'outdir'             : prj_info['outdir'],
        'tprnfor'            : qe_bool(True),
    },
    'system': {
        'ibrav'              : 0,
        'nat'                : len(atoms),
        'ntyp'               : len(set(atoms.get_chemical_symbols())),
        'ecutwfc'            : 20.0, 
        'ecutrho'            : 200.0,
        'occupations'        : 'smearing',
        'smearing'           : 'mp',
        'degauss'            : 0.03,
        'nosym'              : True,
        'assume_isolated'    : 'esm',
        'esm_bc'             : 'bc1',
    },
    'electrons': {
        'mixing_beta'        : 0.3,
    }
}
input_data_chng = {
    prj_info['prefixes'][0]: {
        'control': {
            'prefix'     : prj_info['prefixes'][0],
        }
    }
}
structure_data = extract_structure_data(atoms)
pseudopotentials = {'Al':'Al.pbe-n-van.UPF'}
atomic_species = {
    elem: (atomic_masses[atomic_numbers[elem]], pseudopotentials[elem])
    for elem in pseudopotentials
}

for prefix in prj_info['prefixes']:
    inpfile = f"{prefix}.in"  # 各 prefix ごとに inpfile を作成
    for group in input_data_chng[prefix]:
        for name, value in input_data_chng[prefix][group].items():
            input_data[group][name] = value
            write_qe_input(input_data, atomic_species, structure_data, filename=inpfile, kpts=(8, 8, 1), koffset=(1,1,0))

In [ ]:
#v = nv.show_ase(atoms)
#v.add_ball_and_stick()
#v.add_unitcell()
#v
view(atoms, viewer='ngl')

In [ ]:
# `prj_info` から prefix リストと出力ディレクトリを取得
prefixes = prj_info['prefixes']
outdir = prj_info['outdir']

# Quantum ESPRESSO の XML 解析
scf_result = parse_qe_xml(prefixes, outdir)

# Print all results from XML
for prefix in prj_info['prefixes']:
    print('============================================')
    print(f"Prefix: {prefix}")
    print('============================================')
    # 全エネルギー (eV)
    energy = scf_result[prefix]['energy']
    print(f"Total Energy (eV): {energy:.6f}")
    # フェルミエネルギー (eV)
    fermi_energy = scf_result[prefix]['fermi_energy']
    if fermi_energy is not None:
        print(f"Fermi Energy (eV): {fermi_energy:.6f}")
    # ESM境界条件
    esm_bc = scf_result[prefix]['esm_bc']
    if esm_bc is not None:
        print(f"ESM boundary condition: {esm_bc}")
    # 余剰電荷
    tot_charge = scf_result[prefix]['tot_charge']
    if tot_charge is not None:
        print(f"Total Charge: {tot_charge:.6f}")
    # 原子の力を取得して表示
    forces = scf_result[prefix]['forces']
    if forces:  # フォースデータが空でない場合のみ出力
        print('Forces acting on atoms (eV/Å)')
        for atom_id, fx, fy, fz in forces:
            print(f"  Atom {atom_id}: ({fx:.6f}, {fy:.6f}, {fz:.6f})")
    print('---------------------------------------------')

### Plot `prefix.esm1` file

Now we plot laterally averaged electron density, Hartree, ionic and electrostatic potentials described in  the `prefix.esm1` file that is located under your output directry. 

In [ ]:
plot_esm1(prj_info['outdir']+'/'+prj_info['prefixes'][0]+'.esm1',titlename='Al(111) bc1')

### Work function

Forfurther analysys, we calculate the work function at the Al(111). 
Thanks to Janack's theorem, the work function $\Phi$ can be defined as difference between the vacuum ($E_{\rm vac}$) and Fermi levels ($E_{\rm F}$).  

$$
\Phi = E_{\rm vac} - E_{\rm F}
$$

By using following script, you can extract the $E_{\rm F}$ from the output file of PWscf.  
Try to calculate workfunction using above formula.  

## Al(111) surface ('bc2' boundary condition with external electric field)

Here, we calculate the electronic state at the Al surface under the MSM condition.  
In the MSM, we can apply the external field. Thus, both side of ESM is connected by a lead,   
and the slab is sandwiched by metals.  

Run following python scripts and submit your job to remotehost.  

In [ ]:
project = 'Al-slab'
prj_info = {
    'project': project,
    'projdir': '/Users/otani/Program/q-e',
    'prefixes': ['Al_E'],
}
prj_info['bindir'] = f"{prj_info['projdir']}/bin"
prj_info['pseudodir'] = f"{prj_info['projdir']}/pseudo"
prj_info['outdir'] = "./out"

In [ ]:
job_info = {
    'node':     4,   
    'mpinum':   28,  
    'ompnum':   1,
    'cputime':  dt.timedelta(days=0, hours=0, minutes=30),
    'para_img': 1,
    'para_kpt': 4,
    'para_tsk': 1,
    'para_bnd': 1,
}

### `esm_efield`  

We can find a new input `esm_efield`, which controls a magnitude of applied external field.  
the `esm_efield` is in the unit of [Ry/Bohr] and can be used only in `bc2` condition.  

In [ ]:
atoms = Atoms('Al4', [(2.85595455,  0.00000000,  3.49781569),
                      (1.42797727,  0.82444307,  1.16593856),
                      (2.85595454,  1.64888613, -1.16593856),
                      (0.00000000,  0.00000000, -3.49781569)], 
             cell=[( 2.85595455,  0.0000000000,  0.0000000000),
                   ( 1.42797727,  2.4733291965,  0.0000000000),
                   ( 0.00000000,  0.0000000000, 22.9956313827)],
              pbc=(True, True, False)
)

input_data = {
    'control': {
        'calculation'        : 'scf',
        'restart_mode'       : 'from_scratch',
        'prefix'             : '',
        'max_seconds'        : job_info['cputime'].total_seconds(),
        'pseudo_dir'         : prj_info['pseudodir'],
        'outdir'             : prj_info['outdir'],
        'tprnfor'            : qe_bool(True),
    },
    'system': {
        'ibrav'              : 0,
        'nat'                : len(atoms),
        'ntyp'               : len(set(atoms.get_chemical_symbols())),
        'ecutwfc'            : 20.0, 
        'ecutrho'            : 200.0,
        'occupations'        : 'smearing',
        'smearing'           : 'mp',
        'degauss'            : 0.03,
        'nosym'              : True,
        'assume_isolated'    : 'esm',
        'esm_bc'             : 'bc2',
        'esm_efield'         : 0.01,
    },
    'electrons': {
        'mixing_beta'        : 0.5,
    }
}
input_data_chng = {
    prj_info['prefixes'][0]: {
        'control': {
            'prefix'     : prj_info['prefixes'][0],
        }
    }
}
structure_data = extract_structure_data(atoms)
pseudopotentials = {'Al':'Al.pbe-n-van.UPF'}
atomic_species = {
    elem: (atomic_masses[atomic_numbers[elem]], pseudopotentials[elem])
    for elem in pseudopotentials
}

for prefix in prj_info['prefixes']:
    inpfile = f"{prefix}.in"  # 各 prefix ごとに inpfile を作成
    for group in input_data_chng[prefix]:
        for name, value in input_data_chng[prefix][group].items():
            input_data[group][name] = value
    write_qe_input(input_data, atomic_species, structure_data, filename=inpfile, kpts=(8, 8, 1), koffset=(1,1,0))

### Plot results of ESM calculation with external field

We plot the results of the ESM calculation with exrernal field.  
A lower right panel shows the result of electrostatic potential under external field.  
By the applied external field, the result of electrostatic potential has a slope.  

In [ ]:
plot_esm1(prj_info['outdir']+'/'+prj_info['prefixes'][0]+'.esm1',titlename='Al(111) bc2 with E-field')

## Al(111) surface ( `bc2` boundary condition with various additional charge)

Here, we will calculate electronic structure at Al(111) charged-surface under `bc2` condition.  In ordinary PBC calculation, the electronic structure at a charged-surface is difficult, because electrostatic terms have diverged terms. Usually, charged cell calculation under the PBC is carried out using assumption of uniform jellium-background. However, this treatment cannot determine electrostatic energy, precisely.  

The treatment of charged cell using ESM method allow us to determine electrostatic energy due to image charge induced at ESM surfaces (for further details can be found in Ref. [1]).   

As following, we introduce additional charge using `tot_charge` value.  

In [ ]:
project = 'Al-slab'
prj_info = {
    'project': project,
    'projdir': '/Users/otani/Program/q-e',
    'prefixes':  ['Alm2', 'Alm1', 'Al0', 'Alp1', 'Alp2'],
}
prj_info['bindir'] = f"{prj_info['projdir']}/bin"
prj_info['pseudodir'] = f"{prj_info['projdir']}/pseudo"
prj_info['outdir'] = "./out"

In [ ]:
job_info = {
    'node':     4,   
    'mpinum':   28,  
    'ompnum':   1,
    'cputime':  dt.timedelta(days=0, hours=0, minutes=30),
    'para_img': 1,
    'para_kpt': 4,
    'para_tsk': 1,
    'para_bnd': 1,
}

In [ ]:
atoms = Atoms('Al4', [(2.85595455,  0.00000000,  3.49781569),
                      (1.42797727,  0.82444307,  1.16593856),
                      (2.85595454,  1.64888613, -1.16593856),
                      (0.00000000,  0.00000000, -3.49781569)], 
             cell=[( 2.85595455,  0.0000000000,  0.0000000000),
                   ( 1.42797727,  2.4733291965,  0.0000000000),
                   ( 0.00000000,  0.0000000000, 22.9956313827)],
              pbc=(True, True, False)
)

input_data = {
    'control': {
        'calculation'        : 'scf',
        'restart_mode'       : 'from_scratch',
        'prefix'             : '',
        'max_seconds'        : job_info['cputime'].total_seconds(),
        'pseudo_dir'         : prj_info['pseudodir'],
        'outdir'             : prj_info['outdir'],
        'tprnfor'            : qe_bool(True),
    },
    'system': {
        'ibrav'              : 0,
        'nat'                : len(atoms),
        'ntyp'               : len(set(atoms.get_chemical_symbols())),
        'ecutwfc'            : 20.0, 
        'ecutrho'            : 200.0,
        'occupations'        : 'smearing',
        'smearing'           : 'mp',
        'degauss'            : 0.03,
        'nosym'              : True,
        'assume_isolated'    : 'esm',
        'esm_bc'             : 'bc2',
    },
    'electrons': {
        'diagonalization'    : 'rmm',
        'diago_rmm_conv'     : False,
        'mixing_beta'        : 0.3,
    }
}

input_data_chng = {
    prj_info['prefixes'][0]: {
        'control': {
            'prefix'     : prj_info['prefixes'][0],
        },
        'system' : {
            'tot_charge' : -0.01,
        },
    },
    prj_info['prefixes'][1]: {
        'control': {
            'prefix'     : prj_info['prefixes'][1],
        },
        'system' : {
            'tot_charge' : -0.005,
        },
    },
    prj_info['prefixes'][2]: {
        'control': {
            'prefix'     : prj_info['prefixes'][2],
        },
        'system' : {
            'tot_charge' : 0.0,
        }
    },
    prj_info['prefixes'][3]: {
        'control': {
            'prefix'     : prj_info['prefixes'][3],
        },
        'system' : {
            'tot_charge' : +0.005,
        },
    },
    prj_info['prefixes'][4]: {
        'control': {
            'prefix'     : prj_info['prefixes'][4],
        },
        'system' : {
            'tot_charge' : +0.01,
        }
    },
}
structure_data = extract_structure_data(atoms)
pseudopotentials = {'Al':'Al.pbe-n-van.UPF'}
atomic_species = {
    elem: (atomic_masses[atomic_numbers[elem]], pseudopotentials[elem])
    for elem in pseudopotentials
}

for prefix in prj_info['prefixes']:
    inpfile = f"{prefix}.in"  # 各 prefix ごとに inpfile を作成
    for group in input_data_chng[prefix]:
        for name, value in input_data_chng[prefix][group].items():
            input_data[group][name] = value
    write_qe_input(input_data, atomic_species, structure_data, filename=inpfile, kpts=(8, 8, 1), koffset=(1,1,0))

In [ ]:
# `prj_info` から prefix リストと出力ディレクトリを取得
prefixes = prj_info['prefixes']
outdir = prj_info['outdir']

# Quantum ESPRESSO の XML 解析
scf_result = parse_qe_xml(prefixes, outdir)

# Print all results from XML
for prefix in prj_info['prefixes']:
    print('============================================')
    print(f"Prefix: {prefix}")
    print('============================================')
    # 全エネルギー (eV)
    energy = scf_result[prefix]['energy']
    print(f"Total Energy (eV): {energy:.6f}")
    # フェルミエネルギー (eV)
    fermi_energy = scf_result[prefix]['fermi_energy']
    if fermi_energy is not None:
        print(f"Fermi Energy (eV): {fermi_energy:.6f}")
    # ESM境界条件
    esm_bc = scf_result[prefix]['esm_bc']
    if esm_bc is not None:
        print(f"ESM boundary condition: {esm_bc}")
    # 余剰電荷
    tot_charge = scf_result[prefix]['tot_charge']
    if tot_charge is not None:
        print(f"Total Charge: {tot_charge:.6f}")
    # 原子の力を取得して表示
    forces = scf_result[prefix]['forces']
    if forces:  # フォースデータが空でない場合のみ出力
        print('Forces acting on atoms (eV/Å)')
        for atom_id, fx, fy, fz in forces:
            print(f"  Atom {atom_id}: ({fx:.6f}, {fy:.6f}, {fz:.6f})")
    print('---------------------------------------------')

In [ ]:
view(atoms, viewer='ngl')

### Plot the results of density and potentials at the various charged surface

Here, we plot the results of laterally averaged electronic density and potentials.  
For total charge zero, the results of them are the almost same as those with `bc1` calculation. 

In [ ]:
plot_esm1(prj_info['outdir']+'/'+prj_info['prefixes'][2]+'.esm1',titlename=r'Al with $\pm$0')

In [ ]:
plot_esm1(prj_info['outdir']+'/'+prj_info['prefixes'][0]+'.esm1',titlename='Al with -0.01$|e|$')

In [ ]:
plot_esm1(prj_info['outdir']+'/'+prj_info['prefixes'][4]+'.esm1',titlename='Al with 0.01$|e|$')

### Differential potential

Here, we plot the electrostatic potential difference between non-charged and charged surfaces.  

The induced electric field by additional charge can be clearly seen as followings.  
Furthermore, we found that the potential in the charged slab is almost uniform.  
This result is caused by electron screening effect to the external field.    

In this calculation, unfortunately, induced potential in slab region slightly oscilates.  
This oscilation is caused by insufficient FFT grids. 
If you increase cut-off energy and carry out convergence study, 
this oscilation can be removed. 

In [ ]:
m2 = np.loadtxt(prj_info['outdir']+'/'+prj_info['prefixes'][0]+'.esm1',comments='#')
m1 = np.loadtxt(prj_info['outdir']+'/'+prj_info['prefixes'][1]+'.esm1',comments='#')
n0 = np.loadtxt(prj_info['outdir']+'/'+prj_info['prefixes'][2]+'.esm1',comments='#')
p1 = np.loadtxt(prj_info['outdir']+'/'+prj_info['prefixes'][3]+'.esm1',comments='#')
p2 = np.loadtxt(prj_info['outdir']+'/'+prj_info['prefixes'][4]+'.esm1',comments='#')
fig = plt.figure(
    figsize   = (5,5), # inch
#    dpi       = 100,    # dpi
#    edgecolor = 'black',
#   linewidth = '1'
)
fig.subplots_adjust(wspace=0.3, hspace=0.5,top=0.82)
fig.suptitle('Differential potential')

ax = fig.add_subplot(111)
ax.set_xlabel('z (A)')
ax.set_ylabel('Delta V (eV)')
ax.axhline(0.0, linewidth=1, linestyle='dashed', color='black')
ax.plot(n0[:,0], p2[:,4]-n0[:,4], color='black', linestyle='solid', label='+0.01e')
ax.plot(n0[:,0], m2[:,4]-n0[:,4], color='red', linestyle='solid', label='-0.01e')
ax.plot(n0[:,0], p1[:,4]-n0[:,4], color='blue', linestyle='solid', label='+0.005e')
ax.plot(n0[:,0], m1[:,4]-n0[:,4], color='green', linestyle='solid',label='-0.005e')
plt.show()

## Al(111) surface ('bc3' boundary condition with additional charge [-0.01e])

Here, we learn the vacuum/slab/metal boundary condition calculation.  
Surface slab is sandwiched by the semi-infinite vacuum and metal ESM.  
In this boundary condition, if we set the extra charge, image charge is induced at the metal ESM. 
Thus, electric field is induced only metal side.  

In this calculation, we use `bc3` condition. 

In [ ]:
project = 'Al-slab'
prj_info = {
    'project': project,
    'homedir': os.path.expanduser('~')+'/Program',
    'prefixes':  ['Al111_bc3_m001'],
}

job_info = {
    'node':     4,   
    'mpinum':   28,  
    'ompnum':   1,
    'cputime':  dt.timedelta(days=0, hours=0, minutes=30),
    'para_img': 1,
    'para_kpt': 4,
    'para_tsk': 1,
    'para_bnd': 1,
}

rjob = RemoteJob(prj_info, job_info, remote='enemat')

rjob.connecthost()

In [ ]:
atoms = Atoms('Al4', [(2.85595455,  0.00000000,  3.49781569),
                      (1.42797727,  0.82444307,  1.16593856),
                      (2.85595454,  1.64888613, -1.16593856),
                      (0.00000000,  0.00000000, -3.49781569)], 
             cell=[( 2.85595455,  0.0000000000,  0.0000000000),
                   ( 1.42797727,  2.4733291965,  0.0000000000),
                   ( 0.00000000,  0.0000000000, 22.9956313827)],
              pbc=(True, True, False)
)
pseudo={'Al':'Al.pbe-n-van.UPF'}

input_data = {
    'control': {
        'calculation'        : 'scf',
        'restart_mode'       : 'from_scratch',
        'prefix'             : prefix,
        'pseudo_dir'         : rjob.pseudo,
        'outdir'             : rjob.workdir+'/tmp',
        'tprnfor'            : True,
    },
    'system': {
        'ecutwfc'            : 20.0, 
        'ecutrho'            : 200.0,
        'occupations'        : 'smearing',
        'smearing'           : 'mp',
        'degauss'            : 0.03,
        'nosym'              : True,
        'assume_isolated'    : 'esm',
        'esm_bc'             : 'bc3',
        'tot_charge'         : -0.01
    },
    'electrons': {
        'mixing_beta'        : 0.5,
    }
}

input_data_chng = {
    rjob.prefixes[0]: {
        'control': {
            'prefix'     : rjob.prefixes[0],
        },
    },
}

for prefix in rjob.prefixes:
    for group in input_data_chng[prefix]:
        for name, value in input_data_chng[prefix][group].items():
            input_data[group][name] = value
            write(rjob.srcinp[prefix], atoms, format='espresso-in', input_data=input_data, 
                  pseudopotentials=pseudo, kpts=(8, 8, 1), koffset=(1, 1, 0))

In [ ]:
view(atoms, viewer='ngl')

In [ ]:
rjob.submit_files(prog='pw.x', inpf=rjob.destinp, outf=rjob.destout)

putfiles = [
    rjob.srcsub,
]
putfiles.extend([file for file in rjob.srcinp.values()])
putfiles.extend([file for file in rjob.srcshell.values()])
print('Send files...')
rjob.put_file(putfiles)
for i in range(len(putfiles)):
    print(f"{putfiles[i]} --> {rjob.workdir}")

In [ ]:
rjob.submit_job()

In [ ]:
rjob.remote_qstat()

In [ ]:
done = {}
for prefix in rjob.prefixes:
    done[prefix] = False
print(f"  Job ID    Done")
for prefix in rjob.prefixes:
    print(f"{rjob.jobid[prefix]:^10}{done[prefix]:^10}")
interval = 20
while True:
    print(f"...refresh after {interval} seconds...")
    time.sleep(interval)
    rjob.check_job(done)
    if all(done.values()):
        break
    for prefix in rjob.prefixes:
        print(f"{rjob.jobid[prefix]:^10}{done[prefix]:^10}")
for prefix in rjob.prefixes:
    print(f"{rjob.jobid[prefix]:^10}{done[prefix]:^10}")
print("Job done!")

getfiles = []
getfiles.extend([file for file in rjob.destout.values()])
getfiles.extend([file for file in rjob.destesm1.values()])
print('Get files...')
rjob.get_file(getfiles)
for i in range(len(getfiles)):
    print("{0} --> {1}".format(getfiles[i], rjob.prjhome))

source_dir = rjob.workdir+'/tmp/'+rjob.prefixes[0]+'.save'
rjob.set_base(source_dir, rjob.baseesm)

print("Preparation for esm plot is done.")

### Plot electrostatic potentials

Here, we also plot electrostatic potentials. Origin of electrostatic potential is chosen as the metal ESM surface. 

We clearly found that the electric field is induced for the only side of metal ESM.  

In [ ]:
pt.plot_esm1(rjob.prjhome+'/'+rjob.prefixes[0]+'.esm1',titlename='Al111 bc3 tot_charge=-0.01')

## Al(111) surface ('bc3' boundary condition with constant-mu scheme [target_mu=+1.5 vs PZC])

Here, we learn the how to use the constant-$\mu_{\rm e}$ calculation.   
In this calculation target $\mu_{\rm e}$ is set to +1.5eV comparing to that of  potential zero charge.  
Theoretical back-ground can be learn the published elsewhere [N. Bonnet et al., Phys. Rev. Lett. 109, 266101 (2012).].
This computational technique is very important for investigating the electrochemical interface, because conventional DFT calculation can simulate *only constant-N* calculation. However, in the electrochemical interface, constant-$\mu_{\rm e}$ calculation is required due to the existance of the potentiostat.  

We use `bc3` boundary condition. Thus, Al surface is connected with the metal ESM via the external potentiostat. 

In [ ]:
project = 'Al-slab'
prj_info = {
    'project': project,
    'homedir': os.path.expanduser('~')+'/Program',
    'prefixes':  ['Al111_bc3_vp015'],
}

job_info = {
    'node':     4,   
    'mpinum':   28,  
    'ompnum':   1,
    'cputime':  dt.timedelta(days=0, hours=0, minutes=30),
    'para_img': 1,
    'para_kpt': 4,
    'para_tsk': 1,
    'para_bnd': 1,
}

rjob = RemoteJob(prj_info, job_info, remote='enemat')

rjob.connecthost()

### Input file for constant-$\mu_{\rm e}$ calculation

Here, we briefly describe input values used in constant-$\mu_{\rm e}$ calculation.  
It is noted that the input values are slightly differ from original quantum espresso code.  
More detailed descriptions of inputs can be found in `/QE/PW/Doc/Input_PW.html`.

Below is input creation code with ASE. We can found three new input vaules and one input card:  

* `&FCP` ... This is an input card for contlolling the details of constant-$\mu_{\rm e}$ calculation. This card is activated by `lfcp=.true.`  
* `lfcp` ... Activation flag for constant-$\mu_{\rm e}$ calculation. 
* `fcp_mu` ... This value determine the target $\mu_{\rm e}$ (in eV). 
* `fcp_dynamics` ... dynamics used in the calculation, which should be same as `ion_dynamics` value. 

In addition to the above, we use the `tot_charge = -0.01e`. 


NOTE: to decrease computational costs, we use fixed atomic positions.  

In [ ]:
from ase.constraints import FixAtoms

atoms = Atoms('Al4', [(2.85595455,  0.00000000,  3.49781569),
                      (1.42797727,  0.82444307,  1.16593856),
                      (2.85595454,  1.64888613, -1.16593856),
                      (0.00000000,  0.00000000, -3.49781569)], 
             cell=[( 2.85595455,  0.0000000000,  0.0000000000),
                   ( 1.42797727,  2.4733291965,  0.0000000000),
                   ( 0.00000000,  0.0000000000, 22.9956313827)],
              pbc=(True, True, False)
)
# All Al atoms are
c = FixAtoms(mask=[atom.symbol == 'Al' for atom in atoms])
atoms.set_constraint(c)

pseudo={'Al':'Al.pbe-n-van.UPF'}

input_data = {
    'control': {
        'calculation'        : 'relax',
        'restart_mode'       : 'from_scratch',
        'prefix'             : prefix,
        'pseudo_dir'         : rjob.pseudo,
        'outdir'             : rjob.workdir+'/tmp',
        'tprnfor'            : True,
        'lfcp'               : True,
    },
    'system': {
        'ecutwfc'            : 20.0, 
        'ecutrho'            : 200.0,
        'occupations'        : 'smearing',
        'smearing'           : 'mp',
        'degauss'            : 0.03,
        'nosym'              : True,
        'assume_isolated'    : 'esm',
        'esm_bc'             : 'bc3',
        'tot_charge'         : -0.01,      # Acceralate convergence with good starting tot_charge
    },
    'electrons': {
        'mixing_beta'        : 0.5,
    },
    'ions': {
        'ion_dynamics'       : 'damp',
    },
    'fcp': {
        'fcp_mu'             : -2.5610, # eV -> 1.5 eV above the PZC(=-4.0610) that corresponding to negative value of workfunction at Al(111)
        'fcp_dynamics'       : 'damp',
    }
}

input_data_chng = {
    rjob.prefixes[0]: {
        'control': {
            'prefix'     : rjob.prefixes[0],
        },
    },
}

for prefix in rjob.prefixes:
    for group in input_data_chng[prefix]:
        for name, value in input_data_chng[prefix][group].items():
            input_data[group][name] = value
            write(rjob.srcinp[prefix], atoms, format='espresso-in', input_data=input_data, 
                  pseudopotentials=pseudo, kpts=(8, 8, 1), koffset=(1, 1, 0))

In [ ]:
rjob.submit_files(prog='pw.x', inpf=rjob.destinp, outf=rjob.destout)

putfiles = [
    rjob.srcsub,
]
putfiles.extend([file for file in rjob.srcinp.values()])
putfiles.extend([file for file in rjob.srcshell.values()])
print('Send files...')
rjob.put_file(putfiles)
for i in range(len(putfiles)):
    print(f"{putfiles[i]} --> {rjob.workdir}")

In [ ]:
rjob.submit_job()

In [ ]:
rjob.remote_qstat()

In [ ]:
done = {}
for prefix in rjob.prefixes:
    done[prefix] = False
print(f"  Job ID    Done")
for prefix in rjob.prefixes:
    print(f"{rjob.jobid[prefix]:^10}{done[prefix]:^10}")
interval = 20
while True:
    print(f"...refresh after {interval} seconds...")
    time.sleep(interval)
    rjob.check_job(done)
    if all(done.values()):
        break
    for prefix in rjob.prefixes:
        print(f"{rjob.jobid[prefix]:^10}{done[prefix]:^10}")
for prefix in rjob.prefixes:
    print(f"{rjob.jobid[prefix]:^10}{done[prefix]:^10}")
print("Job done!")

getfiles = []
getfiles.extend([file for file in rjob.destout.values()])
getfiles.extend([file for file in rjob.destesm1.values()])
print('Get files...')
rjob.get_file(getfiles)
for i in range(len(getfiles)):
    print("{0} --> {1}".format(getfiles[i], rjob.prjhome))

source_dir = rjob.workdir+'/tmp/'+rjob.prefixes[0]+'.save'
rjob.set_base(source_dir, rjob.baseesm)

print("Preparation for esm plot is done.")

In [ ]:
pt.plot_esm1(rjob.prjhome+'/'+rjob.prefixes[0]+'.esm1',titlename='Al111 bc3 +1.5 vs PZC')

## Al(111) surface ('bc3' boundary condition with constant-mu scheme [target_mu=-1.5 vs PZC])

In [ ]:
project = 'Al-slab'
prj_info = {
    'project': project,
    'homedir': os.path.expanduser('~')+'/Program',
    'prefixes':  ['Al111_bc3_vp015'],
}

job_info = {
    'node':     4,   
    'mpinum':   28,  
    'ompnum':   1,
    'cputime':  dt.timedelta(days=0, hours=0, minutes=30),
    'para_img': 1,
    'para_kpt': 4,
    'para_tsk': 1,
    'para_bnd': 1,
}

rjob = RemoteJob(prj_info, job_info, remote='enemat')

print('Connecting to RemoteHost...')
rjob.connecthost()

atoms = Atoms('Al4', [(2.85595455,  0.00000000,  3.49781569),
                      (1.42797727,  0.82444307,  1.16593856),
                      (2.85595454,  1.64888613, -1.16593856),
                      (0.00000000,  0.00000000, -3.49781569)], 
             cell=[( 2.85595455,  0.0000000000,  0.0000000000),
                   ( 1.42797727,  2.4733291965,  0.0000000000),
                   ( 0.00000000,  0.0000000000, 22.9956313827)],
              pbc=(True, True, False)
)
#c = FixAtoms(mask=[atom.symbol == 'Al' for atom in atoms])
#atoms.set_constraint(c)

pseudo={'Al':'Al.pbe-n-van.UPF'}

input_data = {
    'control': {
        'calculation'        : 'relax',
        'restart_mode'       : 'from_scratch',
        'prefix'             : prefix,
        'pseudo_dir'         : rjob.pseudo,
        'outdir'             : rjob.workdir+'/tmp',
        'tprnfor'            : True,
        'lfcp'               : True,
    },
    'system': {
        'ecutwfc'            : 20.0, 
        'ecutrho'            : 200.0,
        'occupations'        : 'smearing',
        'smearing'           : 'mp',
        'degauss'            : 0.03,
        'nosym'              : True,
        'assume_isolated'    : 'esm',
        'esm_bc'             : 'bc3',
        'tot_charge'         : 0.01,      # Acceralate convergence with good starting tot_charge
    },
    'electrons': {
        'mixing_beta'        : 0.5,
    },
    'ions': {
        'ion_dynamics'       : 'damp',
    },
    'fcp': {
        'fcp_mu'             : -5.5610, # eV -> 1.5 eV below the PZC(=-4.0610)
        'fcp_dynamics'       : 'damp',
        'freeze_all_atoms'   : True, # Another way to fix the atomic position. See above cell which uses constraint.
    }
}

for prefix in rjob.prefixes:
    for group in input_data_chng[prefix]:
        for name, value in input_data_chng[prefix][group].items():
            input_data[group][name] = value
            write(rjob.srcinp[prefix], atoms, format='espresso-in', input_data=input_data, 
                  pseudopotentials=pseudo, kpts=(8, 8, 1), koffset=(1, 1, 0))
            
print('InputFile generation was done.')

In [ ]:
rjob.submit_files(prog='pw.x', inpf=rjob.destinp, outf=rjob.destout)

putfiles = [
    rjob.srcsub,
]
putfiles.extend([file for file in rjob.srcinp.values()])
putfiles.extend([file for file in rjob.srcshell.values()])
print('Send files...')
rjob.put_file(putfiles)
for i in range(len(putfiles)):
    print(f"{putfiles[i]} --> {rjob.workdir}")
rjob.submit_job()

In [ ]:
rjob.remote_qstat()

In [ ]:
done = {}
for prefix in rjob.prefixes:
    done[prefix] = False
print(f"  Job ID    Done")
for prefix in rjob.prefixes:
    print(f"{rjob.jobid[prefix]:^10}{done[prefix]:^10}")
interval = 20
while True:
    print(f"...refresh after {interval} seconds...")
    time.sleep(interval)
    rjob.check_job(done)
    if all(done.values()):
        break
    for prefix in rjob.prefixes:
        print(f"{rjob.jobid[prefix]:^10}{done[prefix]:^10}")
for prefix in rjob.prefixes:
    print(f"{rjob.jobid[prefix]:^10}{done[prefix]:^10}")
print("Job done!")

getfiles = []
getfiles.extend([file for file in rjob.destout.values()])
getfiles.extend([file for file in rjob.destesm1.values()])
print('Get files...')
rjob.get_file(getfiles)
for i in range(len(getfiles)):
    print("{0} --> {1}".format(getfiles[i], rjob.prjhome))

source_dir = rjob.workdir+'/tmp/'+rjob.prefixes[0]+'.save'
rjob.set_base(source_dir, rjob.baseesm)

print("Preparation for esm plot is done.")

FilePath=rjob.prjhome+'/'+rjob.prefixes[0]+'.esm1'
pt.plot_esm1(FilePath,titlename='Al111 bc3 -1.5 vs PZC')

## Differential potential

In [ ]:
work_path=rjob.prjhome+'/'
bc1 = np.loadtxt(work_path+'Al0.esm1',comments='#')
bc2 = np.loadtxt(work_path+'Al_E.esm1',comments='#')
bc3_vm = np.loadtxt(work_path+'Al111_bc3_m001.esm1')
bc3_vp = np.loadtxt(work_path+'Al111_bc3_vp015.esm1')

fig = plt.figure(
    figsize   = (15, 3), # inch
    dpi       = 100,    # dpi
    edgecolor = 'black',
    linewidth = '1'
)

fig.subplots_adjust(wspace=0.3, hspace=0.5,top=0.82)
fig.suptitle('Differential potential')

ax1 = fig.add_subplot(1,3,1)
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

ax1.set_title('esm/slab/esm')
ax1.set_xlabel('z (A)')
ax1.set_ylabel('Delta V (eV)')
ax2.set_title('vacuum/slab/esm')
ax2.set_xlabel('z (A)')
ax2.set_ylabel('Delta V (eV)')
ax3.set_title('vacuum/slab/esm')
ax3.set_xlabel('z (A)')
ax3.set_ylabel('Delta V (eV)')

ax1.axhline(0.0, linewidth=1, linestyle='dashed', color='black')
ax2.axhline(0.0, linewidth=1, linestyle='dashed', color='black')
ax3.axhline(0.0, linewidth=1, linestyle='dashed', color='black')

ax1.plot(bc2[:,0], bc2[:,4]-bc1[:,4], color='black',linestyle='solid')
ax2.plot(bc3_vm[:,0], bc3_vm[:,4]-bc1[:,4], color='black',linestyle='solid')
ax3.plot(bc3_vp[:,0], bc3_vp[:,4]-bc1[:,4], color='black',linestyle='solid')
#plt.tight_layout()
plt.show()

# Appendix

## Generate Al(111) surface using Materials Project

In [ ]:
%matplotlib inline

from pymatgen import MPRester, Composition, Element
from pymatgen.io.ase import AseAtomsAdaptor
import json
import re
import palettable
import matplotlib as mpl
import nglview
from ase.visualize import view
from ase.build import surface, cut
from pymatgen.io.ase import AseAtomsAdaptor

# atoms is the imported Atoms structure from Materials Project. 

MAPI_KEY = 'bezL4vEqFcOzEGCNFl7'
rester = MPRester(MAPI_KEY)

mp_entries = rester.get_entries('Al', inc_structure=True, sort_by_e_above_hull=True, property_data=['pretty_formula'])
mp_entries[0].entry_id
structure = rester.get_structure_by_material_id(mp_entries[0].entry_id)
print('*** Structure(MP) ***')
print('  ')
print(structure)

atoms = AseAtomsAdaptor.get_atoms(structure)
atoms.pbc=(True, True, False)

#print('  ')
#print('*** Atoms object (ASE) ***')
#print('  ')
#print(atoms)
#view(atoms, viewer='ngl')

orientation = (1, 1, 1)
nlayers = 4
vacuum = 8 #angstroms

s1 = surface(atoms, orientation, nlayers, vacuum=vacuum)
s1.translate((0,0,-(np.sum(s1.positions[:,2]))/s1.positions.shape[0]))
view(s1, viewer='ngl')
#print('{:.3f}'.format(s1.get_positions()))

#
# Expand cell along xy-direction
#

#n1x =  2
#n1y = -2
#n2x =  1
#n2y =  1

# Then cut out the slab
#large_s1 = cut(s1, (n1x,n1y,0), (n2x,n2y,0), (0,0,1) )
#large_s1.translate((0,0,-(np.sum(large_s1.positions[:,2]))/large_s1.positions.shape[0]))
#view(large_s1, viewer='ngl')